In [159]:
# Dependencies
from config import gkey
import pytz
from datetime import datetime
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from pprint import pprint
from geopy.geocoders import Nominatim
from geopy import geocoders 
from scipy.stats import linregress
from matplotlib import pyplot as plt
import os
import json
import urllib.request

# Google developer API key
from config import gkey

In [160]:
accident_data_to_load = "Resources/US_Accidents_May19.csv"
accidents_data_df = pd.read_csv(
    accident_data_to_load)

# Visualize
accidents_data_df = accidents_data_df.reset_index()
accidents_data_df.set_index('ID', inplace=True)
accidents_data_df.head()


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,29.68,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night
A-2,1,OH,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,...,29.65,10.0,Calm,NaN,0.00,Light Rain,False,Night,Night,Day
A-3,2,OH,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,...,29.67,10.0,SW,3.5,NaN,Overcast,True,Night,Night,Day
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day
A-5,4,OH,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,...,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,True,Day,Day,Day


In [161]:
accidents_data_df['Street']

ID
A-1                             I-70 E
A-2                           Brice Rd
A-3                     State Route 32
A-4                             I-75 S
A-5          Miamisburg Centerville Rd
                       ...            
A-1048571               Piney Grove Rd
A-1048572                    Farrow Rd
A-1048573              Moreland Ave SE
A-1048574                 Two Notch Rd
A-1048575               Tidal Point Ln
Name: Street, Length: 1048575, dtype: object

In [162]:
accidents_data_hwy_df = accidents_data_df['Street'].str.split("-", n = 1, expand = True)[1].str.contains("E|W")== True
accidents_data_hwy_df.value_counts()


False    924716
True     123859
Name: 1, dtype: int64

In [163]:
accidents_data_hwy_df.head()

ID
A-1     True
A-2    False
A-3    False
A-4    False
A-5    False
Name: 1, dtype: bool

In [164]:
accidents_data_df['hwy'] = accidents_data_df['Street'].str.split("-", n = 1, expand = True)[1].str.contains("E|W")== True
accidents_data_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-2,1,OH,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,...,10.0,Calm,NaN,0.00,Light Rain,False,Night,Night,Day,False
A-3,2,OH,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,...,10.0,SW,3.5,NaN,Overcast,True,Night,Night,Day,False
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,False
A-5,4,OH,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,...,6.0,SW,3.5,NaN,Mostly Cloudy,True,Day,Day,Day,False


In [165]:
accidents_data_hwy_df = accidents_data_df.loc[accidents_data_df['hwy'] == True]
accidents_data_hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,3.0,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True
A-59,58,OH,2,2/9/16 7:42,2/9/16 8:12,39.754520,-84.159279,0.01,Accident on US-35 Eastbound at Steve Whalen Bl...,NaN,...,1.2,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True
A-60,59,OH,2,2/9/16 7:42,2/9/16 8:12,39.749916,-84.139359,0.01,Accident on US-35 Eastbound at Smithville Rd. ...,NaN,...,1.2,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True
A-69,68,OH,3,2/9/16 8:35,2/9/16 9:05,41.424404,-81.578674,0.01,Accident on I-480 Eastbound at Exit 23 OH-14 B...,NaN,...,10.0,WSW,12.7,NaN,Overcast,False,Day,Day,Day,True


In [166]:
# Output File (CSV)
os.makedirs(os.path.join('output_data'),exist_ok=True)
output_data_file = os.path.join("output_data","accidents_data_hwy_df.csv")
accidents_data_hwy_df.to_csv(output_data_file)

In [167]:
accident_data_hwy_to_load = "Resources/accidents_data_hwy_df.csv"
accidents_hwy_west_df = pd.read_csv(
    accident_data_hwy_to_load, low_memory=False)

# Visualize
accidents_hwy_west_df = accidents_hwy_west_df.reset_index()
accidents_hwy_west_df.set_index('ID', inplace=True)
accidents_hwy_west_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,3.0,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True
A-59,2,58,OH,2,2/9/16 7:42,2/9/16 8:12,39.754520,-84.159279,0.01,Accident on US-35 Eastbound at Steve Whalen Bl...,...,1.2,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True
A-60,3,59,OH,2,2/9/16 7:42,2/9/16 8:12,39.749916,-84.139359,0.01,Accident on US-35 Eastbound at Smithville Rd. ...,...,1.2,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True
A-69,4,68,OH,3,2/9/16 8:35,2/9/16 9:05,41.424404,-81.578674,0.01,Accident on I-480 Eastbound at Exit 23 OH-14 B...,...,10.0,WSW,12.7,NaN,Overcast,False,Day,Day,Day,True


In [168]:
accidents_hwy_west_df['Direction_West'] = accidents_hwy_west_df['Description'].str.contains("Westbound")== True
accidents_hwy_west_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True,False
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,True
A-59,2,58,OH,2,2/9/16 7:42,2/9/16 8:12,39.754520,-84.159279,0.01,Accident on US-35 Eastbound at Steve Whalen Bl...,...,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True,False
A-60,3,59,OH,2,2/9/16 7:42,2/9/16 8:12,39.749916,-84.139359,0.01,Accident on US-35 Eastbound at Smithville Rd. ...,...,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True,False
A-69,4,68,OH,3,2/9/16 8:35,2/9/16 9:05,41.424404,-81.578674,0.01,Accident on I-480 Eastbound at Exit 23 OH-14 B...,...,WSW,12.7,NaN,Overcast,False,Day,Day,Day,True,False


In [169]:
accidents_data_hwy_westbound_df = accidents_hwy_west_df.loc[accidents_hwy_west_df['Direction_West'] == True]
accidents_data_hwy_westbound_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,True
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,WNW,10.4,0.00,Light Snow,False,Day,Day,Day,True,True
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,SSW,4.6,NaN,Scattered Clouds,False,Day,Day,Day,True,True
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,SE,5.8,0.00,Light Freezing Drizzle,False,Night,Night,Night,True,True
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,SSE,8.1,NaN,Light Snow,False,Night,Night,Day,True,True


In [170]:
len(accidents_data_hwy_westbound_df)


52726

In [171]:
accidents_data_hwy_westbound_df['Accident_Date'] = accidents_data_hwy_westbound_df['Start_Time'].str.split(" ", n = 1, expand = True)[0]
accidents_data_hwy_westbound_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,4.6,0.02,Light Rain,False,Day,Day,Day,True,True,2/8/16
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,10.4,0.00,Light Snow,False,Day,Day,Day,True,True,2/10/16
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,4.6,NaN,Scattered Clouds,False,Day,Day,Day,True,True,2/11/16
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,5.8,0.00,Light Freezing Drizzle,False,Night,Night,Night,True,True,2/15/16
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,8.1,NaN,Light Snow,False,Night,Night,Day,True,True,2/15/16


In [172]:
accidents_data_hwy_westbound_df['Accident_Date'] = pd.to_datetime(accidents_data_hwy_westbound_df['Accident_Date'])
accidents_data_hwy_westbound_df['Accident_Date_No_Time'] = accidents_data_hwy_westbound_df['Accident_Date'].dt.date
accidents_data_hwy_westbound_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,0.02,Light Rain,False,Day,Day,Day,True,True,2016-02-08,2016-02-08
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,0.00,Light Snow,False,Day,Day,Day,True,True,2016-02-10,2016-02-10
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,NaN,Scattered Clouds,False,Day,Day,Day,True,True,2016-02-11,2016-02-11
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,0.00,Light Freezing Drizzle,False,Night,Night,Night,True,True,2016-02-15,2016-02-15
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,NaN,Light Snow,False,Night,Night,Day,True,True,2016-02-15,2016-02-15


In [190]:
"""
request format :
https://api.sunrise-sunset.org/json?lat=36.7201600&lng=-4.4203400&date=2019-12-26

"""
base_url = "https://api.sunrise-sunset.org/json?"
latitude = []
longitude = []
date_accident = []
sunset_time = []

for city, state, zipcode, date in zip(accidents_data_hwy_westbound_df.loc[:, 'City'], accidents_data_hwy_westbound_df.loc[:, 'State'], accidents_data_hwy_westbound_df.loc[:, 'Zipcode'], accidents_data_hwy_westbound_df.loc[:, 'Accident_Date_No_Time']):
    
    try:
        #print(city, state, zipcode, date)
        # Target city
        target_city = f"{city}, {state}"

        # Build the endpoint URL
        target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(target_city, gkey)
        #Run a request to endpoint and convert result to json
        geo_data = requests.get(target_url).json()

        # Extract latitude and longitude
        lat = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng = geo_data["results"][0]["geometry"]["location"]["lng"]

        # Print the latitude and longitude
        print('''
             City: {0}
             Latitude: {1}
             Longitude: {2}
              '''.format(target_city, lat, lng))


        sunset_url = f'{base_url}lat={lat}&lng={lng}&date={date}'
        #print(sunset_url)
        response = requests.get(sunset_url, verify=False)
        sunset_data = response.json()
        #pprint(sunset_data)
        print('Sunset:', sunset_data['results']['sunset'])
        print("\n")

        latitude.append(lat)
        longitude.append(lng)
        date_accident.append(date)
        sunset_time.append(sunset_data['results']['sunset'])

    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        print("Connection refused by the server..")
        print("Let me sleep for 5 seconds")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue
    
    
    


             City: Reynoldsburg, OH
             Latitude: 39.9547861
             Longitude: -82.81211909999999
              
Sunset: 10:58:48 PM




/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



             City: Barberton, OH
             Latitude: 41.012833
             Longitude: -81.6051221
              
Sunset: 10:54:32 PM




/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



             City: Dayton, OH
             Latitude: 39.7589478
             Longitude: -84.1916069
              
Sunset: 11:08:13 PM




...

Sunset: 1:36:46 AM



             City: Richmond, CA
             Latitude: 37.9357576
             Longitude: -122.3477486
              


/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Sunset: 1:40:37 AM



             City: Oakland, CA
             Latitude: 37.8043514
             Longitude: -122.2711639
              


/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


AttributeError: 'dict' object has no attribute 'status_code'

In [191]:
print(sunset_time)
len(sunset_time)

['10:58:48 PM', '10:54:32 PM', '11:08:13 PM', '11:07:50 PM', '11:07:50 PM', '11:09:00 PM', '11:06:15 PM', '11:13:49 PM', '11:31:57 PM', '11:28:07 PM', '11:34:32 PM', '11:37:55 PM', '11:40:23 PM', '3:32:42 AM', '3:34:34 AM', '3:34:17 AM', '3:34:00 AM', '3:32:23 AM', '3:33:50 AM', '3:34:43 AM', '3:31:08 AM', '3:34:43 AM', '3:35:16 AM', '3:31:08 AM', '3:33:42 AM', '3:33:10 AM', '3:34:10 AM', '3:35:35 AM', '3:35:36 AM', '3:31:18 AM', '3:35:36 AM', '3:34:10 AM', '3:33:29 AM', '3:35:03 AM', '3:34:10 AM', '3:35:19 AM', '3:35:27 AM', '3:34:18 AM', '3:35:10 AM', '3:35:27 AM', '3:32:04 AM', '3:35:01 AM', '3:32:38 AM', '3:32:38 AM', '3:35:47 AM', '3:33:51 AM', '3:35:07 AM', '3:35:49 AM', '3:35:07 AM', '3:35:49 AM', '3:35:20 AM', '3:35:07 AM', '3:34:24 AM', '3:35:18 AM', '3:32:44 AM', '3:33:56 AM', '3:34:15 AM', '3:35:16 AM', '3:35:20 AM', '3:35:07 AM', '3:31:37 AM', '3:35:54 AM', '3:35:58 AM', '3:34:30 AM', '3:35:38 AM', '3:35:12 AM', '3:34:20 AM', '3:33:13 AM', '3:34:33 AM', '3:35:45 AM', '3:35:

2122

In [192]:
accidents_data_hwy_westbound_sunset_df = accidents_data_hwy_westbound_df.iloc[:len(sunset_time), :]
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,0.02,Light Rain,False,Day,Day,Day,True,True,2016-02-08,2016-02-08
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,0.00,Light Snow,False,Day,Day,Day,True,True,2016-02-10,2016-02-10
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,NaN,Scattered Clouds,False,Day,Day,Day,True,True,2016-02-11,2016-02-11
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,0.00,Light Freezing Drizzle,False,Night,Night,Night,True,True,2016-02-15,2016-02-15
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,NaN,Light Snow,False,Night,Night,Day,True,True,2016-02-15,2016-02-15


In [193]:
len(accidents_data_hwy_westbound_sunset_df)

2122

In [200]:
accidents_data_hwy_westbound_sunset_df['Sunset_Time_UTC'] = sunset_time
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,Light Rain,False,Day,Day,Day,True,True,10:58:48 PM,8:21,22:58:48
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,Light Snow,False,Day,Day,Day,True,True,10:54:32 PM,17:10,22:54:32
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,Scattered Clouds,False,Day,Day,Day,True,True,11:08:13 PM,8:13,23:08:13
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,Light Freezing Drizzle,False,Night,Night,Night,True,True,11:07:50 PM,5:42,23:07:50
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,Light Snow,False,Night,Night,Day,True,True,11:07:50 PM,6:36,23:07:50


In [201]:
accidents_data_hwy_westbound_sunset_df = accidents_data_hwy_westbound_sunset_df.rename({'Accident_Date_No_Time':'Accident_Date'}, axis=1)
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,Light Rain,False,Day,Day,Day,True,True,10:58:48 PM,8:21,22:58:48
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,Light Snow,False,Day,Day,Day,True,True,10:54:32 PM,17:10,22:54:32
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,Scattered Clouds,False,Day,Day,Day,True,True,11:08:13 PM,8:13,23:08:13
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,Light Freezing Drizzle,False,Night,Night,Night,True,True,11:07:50 PM,5:42,23:07:50
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,Light Snow,False,Night,Night,Day,True,True,11:07:50 PM,6:36,23:07:50


In [202]:
accidents_data_hwy_westbound_sunset_df['Accident_Time'] = accidents_data_hwy_westbound_sunset_df['Start_Time'].str.split(" ", n = 1, expand = True)[1]
accidents_data_hwy_westbound_sunset_df['Accident_Date'] = accidents_data_hwy_westbound_sunset_df['Start_Time'].str.split(" ", n = 1, expand = True)[0]
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Accident_Date
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,False,Day,Day,Day,True,True,10:58:48 PM,8:21,22:58:48,2/8/16
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,False,Day,Day,Day,True,True,10:54:32 PM,17:10,22:54:32,2/10/16
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,False,Day,Day,Day,True,True,11:08:13 PM,8:13,23:08:13,2/11/16
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,False,Night,Night,Night,True,True,11:07:50 PM,5:42,23:07:50,2/15/16
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,False,Night,Night,Day,True,True,11:07:50 PM,6:36,23:07:50,2/15/16


In [203]:
accidents_data_hwy_westbound_sunset_df['Accident_Date'] = accidents_data_hwy_westbound_sunset_df['Accident_Date'].str.replace('/','-')
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Accident_Date
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,False,Day,Day,Day,True,True,10:58:48 PM,8:21,22:58:48,2-8-16
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,False,Day,Day,Day,True,True,10:54:32 PM,17:10,22:54:32,2-10-16
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,False,Day,Day,Day,True,True,11:08:13 PM,8:13,23:08:13,2-11-16
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,False,Night,Night,Night,True,True,11:07:50 PM,5:42,23:07:50,2-15-16
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,False,Night,Night,Day,True,True,11:07:50 PM,6:36,23:07:50,2-15-16


In [204]:
conditions = [
    (accidents_data_hwy_westbound_sunset_df['State'] == 'OH'),
    (accidents_data_hwy_westbound_sunset_df['State'] == 'CA'),
]
choices = ['US/Eastern', 'US/Pacific']
accidents_data_hwy_westbound_sunset_df['Timezone'] = np.select(conditions, choices, default='US/Eastern')
accidents_data_hwy_westbound_sunset_df.head()

#accidents_data_hwy_westbound_sunset_df['Timezone'] = np.where(accidents_data_hwy_westbound_sunset_df['State']=='OH', 'US/Eastern', 'US/Pacific')
#accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Accident_Date
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,False,Day,Day,Day,True,True,10:58:48 PM,8:21,22:58:48,2-8-16
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,False,Day,Day,Day,True,True,10:54:32 PM,17:10,22:54:32,2-10-16
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,False,Day,Day,Day,True,True,11:08:13 PM,8:13,23:08:13,2-11-16
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,False,Night,Night,Night,True,True,11:07:50 PM,5:42,23:07:50,2-15-16
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,False,Night,Night,Day,True,True,11:07:50 PM,6:36,23:07:50,2-15-16


In [205]:
accidents_data_hwy_westbound_sunset_df['Sunset_Time_UTC_Hours'] = pd.to_datetime(accidents_data_hwy_westbound_sunset_df['Sunset_Time_UTC']).dt.strftime('%H:%M:%S')
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Accident_Date
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,False,Day,Day,Day,True,True,10:58:48 PM,8:21,22:58:48,2-8-16
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,False,Day,Day,Day,True,True,10:54:32 PM,17:10,22:54:32,2-10-16
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,False,Day,Day,Day,True,True,11:08:13 PM,8:13,23:08:13,2-11-16
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,False,Night,Night,Night,True,True,11:07:50 PM,5:42,23:07:50,2-15-16
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,False,Night,Night,Day,True,True,11:07:50 PM,6:36,23:07:50,2-15-16


In [206]:
accidents_data_hwy_westbound_sunset_date_time_df = accidents_data_hwy_westbound_sunset_df.assign(Sunset_Date_Time = accidents_data_hwy_westbound_sunset_df.Accident_Date.astype(str) + ' ' + \
  accidents_data_hwy_westbound_sunset_df.Sunset_Time_UTC_Hours.astype(str) )
accidents_data_hwy_westbound_sunset_date_time_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Accident_Date,Sunset_Date_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,Day,Day,Day,True,True,10:58:48 PM,8:21,22:58:48,2-8-16,2-8-16 22:58:48
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,Day,Day,Day,True,True,10:54:32 PM,17:10,22:54:32,2-10-16,2-10-16 22:54:32
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,Day,Day,Day,True,True,11:08:13 PM,8:13,23:08:13,2-11-16,2-11-16 23:08:13
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,Night,Night,Night,True,True,11:07:50 PM,5:42,23:07:50,2-15-16,2-15-16 23:07:50
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,Night,Night,Day,True,True,11:07:50 PM,6:36,23:07:50,2-15-16,2-15-16 23:07:50


In [211]:
for date, time, timezone in zip(accidents_data_hwy_westbound_sunset_date_time_df.loc[:, 'Accident_Date'], accidents_data_hwy_westbound_sunset_date_time_df.loc[:, 'Sunset_Time_UTC_Hours'], accidents_data_hwy_westbound_sunset_date_time_df.loc[:, 'Timezone']):
    #print(date, time, timezone)  
    est = pytz.timezone('US/Eastern')
    pst = pytz.timezone('US/Pacific')
    utc = pytz.utc
    fmt = '%Y-%m-%d %H:%M:%S %Z%z'

    utc_time = datetime(int('20'+ str(date).split('-')[2]), int(str(date).split('-')[0]), int(str(date).split('-')[1]), int(str(time).split(':')[0]), int(str(time).split(':')[1]), int(str(time).split(':')[2]), tzinfo=utc)
    
    if (timezone == 'US/Eastern'):
        print(utc_time.astimezone(est).strftime(fmt))
    elif (timezone == 'US/Pacific'):
        print(utc_time.astimezone(pst).strftime(fmt))
    


2016-02-08 17:58:48 EST-0500
2016-02-10 17:54:32 EST-0500
2016-02-11 18:08:13 EST-0500
2016-02-15 18:07:50 EST-0500
2016-02-15 18:07:50 EST-0500
2016-02-16 18:09:00 EST-0500
2016-02-17 18:06:15 EST-0500
2016-02-26 18:13:49 EST-0500
2016-03-03 18:31:57 EST-0500
2016-03-04 18:28:07 EST-0500
2016-03-10 18:34:32 EST-0500
2016-03-15 19:37:55 EDT-0400
2016-03-16 19:40:23 EDT-0400
2016-06-20 20:32:42 PDT-0700
2016-06-20 20:34:34 PDT-0700
2016-06-20 20:34:17 PDT-0700
2016-06-20 20:34:00 PDT-0700
2016-06-20 20:32:23 PDT-0700
2016-06-20 20:33:50 PDT-0700
2016-06-20 20:34:43 PDT-0700
2016-06-20 20:31:08 PDT-0700
2016-06-20 20:34:43 PDT-0700
2016-06-20 20:35:16 PDT-0700
2016-06-20 20:31:08 PDT-0700
2016-06-21 20:33:42 PDT-0700
2016-06-21 20:33:10 PDT-0700
2016-06-21 20:34:10 PDT-0700
2016-06-21 20:35:35 PDT-0700
2016-06-21 20:35:36 PDT-0700
2016-06-21 20:31:18 PDT-0700
2016-06-21 20:35:36 PDT-0700
2016-06-21 20:34:10 PDT-0700
2016-06-21 20:33:29 PDT-0700
2016-06-21 20:35:03 PDT-0700
2016-06-21 20:

2016-09-28 18:54:30 PDT-0700
2016-09-28 18:51:55 PDT-0700
2016-09-28 18:53:54 PDT-0700
2016-09-28 18:52:17 PDT-0700
2016-09-28 18:52:17 PDT-0700
2016-09-28 18:53:54 PDT-0700
2016-09-28 18:53:54 PDT-0700
2016-09-28 18:50:34 PDT-0700
2016-09-28 18:52:50 PDT-0700
2016-09-28 18:53:49 PDT-0700
2016-09-28 18:53:54 PDT-0700
2016-09-28 18:53:58 PDT-0700
2016-09-28 18:51:55 PDT-0700
2016-09-28 18:54:10 PDT-0700
2016-09-28 18:54:10 PDT-0700
2016-09-28 18:50:43 PDT-0700
2016-09-28 18:52:53 PDT-0700
2016-09-29 18:51:34 PDT-0700
2016-09-29 18:47:44 PDT-0700
2016-09-29 18:51:45 PDT-0700
2016-09-29 18:52:25 PDT-0700
2016-09-29 18:52:22 PDT-0700
2016-09-29 18:50:59 PDT-0700
2016-09-29 18:50:50 PDT-0700
2016-09-29 18:52:58 PDT-0700
2016-09-29 18:49:01 PDT-0700
2016-09-29 18:52:26 PDT-0700
2016-09-29 18:49:01 PDT-0700
2016-09-29 18:50:57 PDT-0700
2016-09-30 18:50:51 PDT-0700
2016-09-30 18:50:13 PDT-0700
2016-09-30 18:49:13 PDT-0700
2016-09-30 18:50:51 PDT-0700
2016-09-30 18:51:06 PDT-0700
2016-09-30 18: